## Imports

In [ ]:
import re
import os
import glob
import random
import pandas as pd
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.decomposition import KernelPCA, TruncatedSVD
from sklearn.model_selection import StratifiedKFold
from sklearn.pipeline import Pipeline
import xgboost as xgb
from genetic_algorithm import GeneticAlgorithmCV
from concurrent.futures import ThreadPoolExecutor, wait
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import convert_sklearn, update_registered_converter
from skl2onnx.common.shape_calculator import calculate_linear_classifier_output_shapes
from onnxmltools.convert.xgboost.operator_converters.XGBoost import convert_xgboost
import warnings
warnings.filterwarnings('ignore')

## Functions

In [ ]:
# Helper function
def process_files(file_paths):
    data_list = []
    for file_path in file_paths:
        model_type = extract_action(file_path)
        df = pd.read_csv(file_path)

        # (1) generas X_train, y_train con create_training_dataset
        X_train, y_train = create_training_dataset(df, model_type)

        # (2) filtras filas conflictivas (la lógica avanzada)
        X_clean, y_clean = meta_label_data_multi_bootstrap_oob(
            X_train, y_train,
            models_number=10,         # o el que quieras
            bad_samples_fraction=0.8  # ajusta a tu criterio
        )

        data_list.append((X_clean, y_clean, model_type))
    return data_list

# Helper function
def extract_action(filepath):
    match = re.search(r'(long|short)', filepath)
    return match.group(0) if match else None

# Crear datasets de entrenamiento
def create_training_dataset(df, action_type):
    print(f"=== Procesando dataset de {'compras' if action_type == 'long' else 'ventas'} ===")
    df = df.drop_duplicates()
    print(f"Total de operaciones después de eliminar duplicados: {len(df)}")
    # Filtrar las operaciones con profit != 0
    df_trade = df[df['profit'] != 0].copy()
    print(f"Operaciones con profit != 0: {len(df_trade)}")
    # Añadir la columna 'target' basada en el profit
    df_trade['target'] = df_trade['profit'].apply(lambda x: 1 if x > 0 else 0)
    num_ganadoras = df_trade['target'].sum()
    num_perdedoras = len(df_trade) - num_ganadoras
    print(f"Operaciones ganadoras: {int(num_ganadoras)}")
    print(f"Operaciones perdedoras: {int(num_perdedoras)}")
    # Eliminar posibles missings
    if df_trade.isna().values.any():
        num_missings = df_trade.isna().sum().sum()
        print(f"Valores faltantes encontrados: {num_missings}")
        df_trade = df_trade.dropna()
        print(f"Total de operaciones después de eliminar missings: {len(df_trade)}")
    df_training = df_trade.copy(deep=True)
    # Seleccionar las columnas necesarias (todas menos las dos últimas para el conjunto principal)
    feature_columns = df_training.columns[:-2]
    df_training = df_training[feature_columns.tolist() + ['target']]
    # Preparación de los datos de entrenamiento
    X_train = df_training.drop(columns='target').values.astype('float32')
    y_train = df_training['target'].values.astype('float32')
    print(f"Dataset final preparado: {X_train.shape[0]} operaciones, {X_train.shape[1]} características")
    return X_train, y_train

def sample_random_hparams():
    """
    Retorna un dict con hiperparámetros aleatorios de XGBoost 
    dentro de rangos razonables. Ajusta a tu gusto.
    """
    return {
        'n_estimators':     random.randint(50, 500),
        'max_depth':        random.randint(3, 10),
        'eta':              random.uniform(0.1, 0.5),
        'gamma':            random.uniform(0.0, 0.5),
        'subsample':        random.uniform(0.5, 1.0),
        'colsample_bytree': random.uniform(0.5, 1.0),
        'colsample_bylevel':random.uniform(0.5, 1.0),
        'colsample_bynode': random.uniform(0.5, 1.0),
        'alpha':            random.uniform(0.0, 1.0),
        'lambda':           random.uniform(0.0, 1.0),
        'min_child_weight': random.randint(1, 10),
        'scale_pos_weight': random.randint(1, 10)
    }

def meta_label_data_multi_bootstrap_oob(
    X, y, 
    models_number=5, 
    bad_samples_fraction=0.8
):
    """
    Versión 'avanzada' de meta-labeling con bootstrapping y uso de OOB.
    - Cada iteración:
      (a) Sample (frac=0.5, replace=True) => train_sample
      (b) OOB = filas que quedaron fuera => val_sample
      (c) Entrena en train_sample
      (d) Predice solo en val_sample
      (e) Acumula las filas de val_sample mal clasificadas
    - Al final, elimina filas que superan el umbral (mean*bad_samples_fraction)
    """
    df = pd.DataFrame(X)
    df['target'] = y
    
    # Índices de malas muestras
    BAD_0 = pd.Index([])
    BAD_1 = pd.Index([])

    for i in range(models_number):
        # (1) Generar muestra bootstrap (train_sample)
        train_sample = df.sample(frac=0.5, replace=True, random_state=None)
        
        # (2) Conjunto OOB = val_sample
        val_sample = df.loc[~df.index.isin(train_sample.index)]

        # (3) Hiperparámetros (aleatorios o fijos)
        hparams = sample_random_hparams()
        model = xgb.XGBClassifier(
            use_label_encoder=False, 
            eval_metric='logloss',
            **hparams
        )

        # (4) Entrenar en la parte bootstrap
        model.fit(
            train_sample.drop(columns='target'),
            train_sample['target']
        )

        # (5) Predecir en OOB
        if len(val_sample) == 0:
            # Si en algún caso random la muestra bootstrap coge todo, esto evitará un error
            continue

        pred_proba = model.predict_proba(val_sample.drop(columns='target'))[:, 1]
        pred_labels = (pred_proba >= 0.5).astype(int)

        # (6) Identificar malas muestras solo en val_sample
        val_sample = val_sample.copy()
        val_sample['pred'] = pred_labels
        
        val_sample_0 = val_sample[val_sample['target'] == 0]
        val_sample_1 = val_sample[val_sample['target'] == 1]
        
        diff_0 = val_sample_0.index[val_sample_0['target'] != val_sample_0['pred']]
        diff_1 = val_sample_1.index[val_sample_1['target'] != val_sample_1['pred']]

        # (7) Acumular esos índices
        BAD_0 = BAD_0.append(diff_0)
        BAD_1 = BAD_1.append(diff_1)

    # (8) Contar la frecuencia de error de cada índice
    to_mark_0 = BAD_0.value_counts()
    to_mark_1 = BAD_1.value_counts()

    # (9) Definir umbral
    threshold_0 = to_mark_0.mean() * bad_samples_fraction if len(to_mark_0) else 0
    threshold_1 = to_mark_1.mean() * bad_samples_fraction if len(to_mark_1) else 0

    marked_0 = to_mark_0[to_mark_0 > threshold_0].index if len(to_mark_0) else []
    marked_1 = to_mark_1[to_mark_1 > threshold_1].index if len(to_mark_1) else []

    all_bad = pd.Index(marked_0).union(marked_1)
    
    # (10) Filtrar las filas marcadas del dataset completo
    good_mask = ~df.index.isin(all_bad)
    df_filtered = df.loc[good_mask].copy()

    print(f"[meta_label_data_multi_bootstrap_oob] Filas totales={len(df)}, removidas={len(df)-len(df_filtered)}, restantes={len(df_filtered)}")

    X_filtered = df_filtered.drop(columns='target').values
    y_filtered = df_filtered['target'].values
    return X_filtered, y_filtered

# Train model functions
def train_classifier(X_train, y_train, model_type):
    n_features = X_train.shape[1]
    # Definir algoritmo de validación cruzada
    cv = StratifiedKFold(n_splits=3)
    # Definir Pipeline
    pipeline = Pipeline([
            ('scaler', 'passthrough'),
            ('reducer', 'passthrough'),
            ('xgbclassifier', xgb.XGBClassifier(
                device = "cuda",
                verbosity=0)
            )
        ])
    # Definir mapa de estimadores
    estimator_map = {
        'scaler': {
            'standard': StandardScaler(),
            'robust': RobustScaler(),
            'none': 'passthrough'
        },
        'reducer': {
            'kernel_pca_rbf': KernelPCA(kernel='rbf'),
            'kernel_pca_linear': KernelPCA(kernel='linear'),
            'truncated_svd': TruncatedSVD(),
            'none': 'passthrough'
        }
    }
    # Definir espacio de hiperparámetros para compras
    param_grid = {
        'scaler': {'type': 'categorical', 'values': ['standard', 'robust']},
        'reducer': {'type': 'categorical', 'values': [
            'kernel_pca_rbf', 'kernel_pca_linear', 'truncated_svd']},
        'reducer__n_components': {'type': 'int', 'low': 2, 'high': n_features-1},
        'xgbclassifier__n_estimators': {'type': 'int', 'low': 50, 'high': 500},
        'xgbclassifier__max_depth': {'type': 'int', 'low': 3, 'high': 10},
        'xgbclassifier__eta':{'type': 'float', 'low': 0.1, 'high': 0.5},
        'xgbclassifier__gamma':{'type': 'float', 'low': 0.0, 'high': 0.5},
        'xgbclassifier__subsample': {'type': 'float', 'low': 0.5, 'high': 1.0},
        'xgbclassifier__colsample_bytree': {'type': 'float', 'low': 0.5, 'high': 1.0},
        'xgbclassifier__colsample_bylevel': {'type': 'float', 'low': 0.5, 'high': 1.0},
        'xgbclassifier__colsample_bynode': {'type': 'float', 'low': 0.5, 'high': 1.0},
        'xgbclassifier__alpha': {'type': 'float', 'low': 0.0, 'high': 1.0},
        'xgbclassifier__lambda': {'type': 'float', 'low': 0.0, 'high': 1.0},
        'xgbclassifier__min_child_weight': {'type': 'int', 'low': 1, 'high': 10},
        'xgbclassifier__scale_pos_weight':  {'type': 'int', 'low': 1, 'high': 10}
    }
    try:
        # Entrenar el modelo utilizando el algoritmo genético
        ga_search = GeneticAlgorithmCV(
            model_type=model_type,
            pipeline=pipeline,
            param_grid=param_grid,
            estimator_map=estimator_map,
            cv=cv,
            pop_size=45,
            generations=15,
            early_stopping_rounds=3,
            crossover_initial=0.9,
            crossover_end=0.1,
            mutation_initial=0.1,
            mutation_end=0.9,
            elitism=True,
            elite_size=5,
            tournament_size=3,
            n_random=5,
            n_jobs=2,
            verbose=True,
        )
        ga_search.fit(X_train, y_train)
    except Exception as e:
        print(f"Error en traing model {model_type}: {e}")
        raise
    # Obtener los mejores parámetros y el mejor estimador
    print("####################################################################")
    print(f"Mejor puntuación de validación para {model_type}: {ga_search.best_score_}")
    print(f"Mejores parámetros encontrados para {model_type}: {ga_search.best_params_full_}")
    print("####################################################################")
    # Retornar mejor estimador
    return ga_search.best_estimator_

# Guardar modelos en formato ONNX
def save_onnx_model(mql5_files_folder, model, X, model_type):
    try:
        update_registered_converter(
            xgb.XGBClassifier,
            "XGBClassifier",
            calculate_linear_classifier_output_shapes,
            convert_xgboost,
            options={'nocl': [True, False], 'zipmap': [True, False, 'columns']}
        )
        model_onnx = convert_sklearn(
            model,
            f"pipeline_{model_type}_xgboost",
            [('input', FloatTensorType([None, X.shape[1]]))],
            target_opset={'': 12, 'ai.onnx.ml': 2}
        )
        with open(os.path.join(mql5_files_folder, f"model_{model_type}.onnx"), 'wb') as f:
            f.write(model_onnx.SerializeToString())
    except Exception as e:
        print(f"Error en exportar el modelo {model_type}: {e}")
        raise
    print(f"Modelo {model_type} ONNX exportado correctamente")

## Train

In [ ]:
# Rutas
common_file_folder = r"/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/Common/Files/"
mql5_files_folder = r'/mnt/c/Users/Administrador/AppData/Roaming/MetaQuotes/Terminal/6C3C6A11D1C3791DD4DBF45421BF8028/MQL5/Files/'
# Definir patrones de archivos para compras y ventas
long_file_pattern = os.path.join(common_file_folder, 'training_dataset_long_*.csv')
short_file_pattern = os.path.join(common_file_folder, 'training_dataset_short_*.csv')
generic_file_pattern = os.path.join(common_file_folder, 'training_dataset_*.csv')
# Encontrar archivos
df_long_file_paths = glob.glob(long_file_pattern)
df_short_file_paths = glob.glob(short_file_pattern)
df_generic_file_paths = glob.glob(generic_file_pattern)
# Inicializar una lista para almacenar los datos
data_list = []
# Procesar archivos de compras
if df_long_file_paths:
    data_list.extend(process_files(df_long_file_paths))
# Procesar archivos de ventas
if df_short_file_paths:
    data_list.extend(process_files(df_short_file_paths))
# Si no se encontraron archivos específicos de compras o ventas, buscar archivos genéricos
if not df_long_file_paths and not df_short_file_paths:
    if df_generic_file_paths:
        data_list.extend(process_files(df_generic_file_paths))
# Verificar cuántos conjuntos de datos tenemos y proceder en consecuencia
if len(data_list) == 0:
    print("No se encontraron archivos de datos para entrenar.")
elif len(data_list) == 1:
    # Solo hay un conjunto de datos
    X_train, y_train, model_type = data_list[0]
    print(f"Entrenando el modelo clasificador para {model_type}...")
    model = train_classifier(X_train, y_train, model_type)
    print("¡Modelo entrenado!")
    # Guardar el modelo entrenado en formato ONNX
    save_onnx_model(mql5_files_folder, model, X_train, model_type)
else:
    # Hay múltiples conjuntos de datos, entrenar en paralelo
    with ThreadPoolExecutor(max_workers=len(data_list)) as executor:
        print("Entrenando modelos en paralelo...")
        futures = []
        for X_train, y_train, model_type in data_list:
            future = executor.submit(train_classifier, X_train, y_train, model_type)
            futures.append((future, X_train, model_type))
        # Esperar a que todas las tareas terminen
        wait([f[0] for f in futures])
        # Obtener resultados y guardar modelos
        for future, X_train, model_type in futures:
            model = future.result()
            print(f"¡Modelo para {model_type} entrenado!")
            # Guardar el modelo entrenado en formato ONNX
            save_onnx_model(mql5_files_folder, model, X_train, model_type)

##########
